In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [47]:
pib = pd.read_csv("C:/Users/SURFACE/Documents/GitHub/Python-data-science-Lecalve-Majka/Donnees_python_data_science/donnees_dezippe/augmentation_pib/chiffre_augment_pib.csv")
emission_GES = pd.read_csv("C:/Users/SURFACE/Documents/Ensae/2A/PDS/Projet/Donnees_python_data_science/donnees_dezippe/GES_total/chiffre_GES_total.csv", skiprows = [0,3])

In [48]:
pib.head(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.369863,4.198232,0.300000,5.700001,2.100000,1.999999,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,12.752287,5.600745,2.724543,1.451315,2.260314,2.665292,1.840089,2.901229,NaN,NaN
2,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,8.542147,4.954590,4.822626,0.943576,-2.580097,-0.147207,-2.003577,-0.869394,NaN,NaN
3,Albania,ALB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.417526,1.001987,1.774487,2.218752,3.314805,3.802197,4.071301,2.214063,NaN,NaN
4,Andorra,AND,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.974444,-3.547597,2.504466,1.434140,3.709678,0.346072,1.588765,1.849238,NaN,NaN
5,Arab World,ARB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,6.590121,3.184266,2.447410,3.173547,3.421765,1.137169,2.089837,1.478440,NaN,NaN
6,United Arab Emirates,ARE,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.484626,5.053346,4.284304,5.105937,3.060964,2.373537,1.189856,1.677720,NaN,NaN
7,Argentina,ARG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,5.427843,-0.852022,-5.308197,10.130298,10.569433,...,-1.026420,2.405324,-2.512615,2.731160,-2.080328,2.668590,-2.481792,-2.162952,NaN,NaN
8,Armenia,ARM,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,7.200000,3.300000,3.600000,3.200000,0.200000,7.500000,5.200000,7.600000,NaN,NaN
9,American Samoa,ASM,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.392387,-2.756508,0.944882,1.404056,-2.769231,-5.854430,2.184874,NaN,NaN,NaN


In [43]:
emission_GES.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Last Updated Date,2020-10-15,Unnamed: 2
Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.000000,1971.000000,1972.000000,1973.00000,1974.000000,1975.000000,1976.000000,1977.000000,1978.000000,1979.000000,1980.000000,1981.000000,1982.000000,1983.000000,1984.000000,1985.000000,1986.00000,1987.000000,1.988000e+03,1.989000e+03,1.990000e+03,1.991000e+03,1.992000e+03,1.993000e+03,1.994000e+03,1.995000e+03,1.996000e+03,1.997000e+03,1.998000e+03,1.999000e+03,2.000000e+03,2.001000e+03,2.002000e+03,2.003000e+03,2.004000e+03,2.005000e+03,2.006000e+03,2.007000e+03,2.008000e+03,2.009000e+03,2.010000e+03,2.011000e+03,2.012000e+03,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,NaN
Aruba,ABW,Émissions totales de GES (kt d’équivalent CO2),EN.ATM.GHGT.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.306298,42.786948,43.286613,43.72459,44.130957,44.481908,131.633746,135.055319,358.990001,409.505485,424.783771,519.001013,564.023574,530.385687,571.038292,642.147964,435.85886,616.928696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,AFG,Émissions totales de GES (kt d’équivalent CO2),EN.ATM.GHGT.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14306.616300,14391.778200,13040.848000,13535.75410,14945.968200,14574.164000,14746.851000,15247.281000,15112.318000,14923.075000,15691.266000,15810.335000,15615.333000,15322.530000,14278.792100,13545.736900,12269.46240,12400.507300,1.295929e+04,1.234896e+04,1.252802e+04,1.230706e+04,1.137045e+04,1.139087e+04,1.129069e+04,1.163566e+04,1.250593e+04,1.334325e+04,1.379337e+04,1.481237e+04,1.353095e+04,1.258540e+04,1.384274e+04,1.470988e+04,1.465437e+04,1.477026e+04,1.484916e+04,1.527263e+04,1.528878e+04,1.555443e+04,1.779133e+04,1.798131e+04,1.816886e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,AGO,Émissions totales de GES (kt d’équivalent CO2),EN.ATM.GHGT.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60648.601080,39162.171430,42205.650700,44252.66912,40179.233500,44429.300030,38448.024090,43563.412180,46235.114580,56644.651970,48001.866160,41894.794960,45243.802080,44852.060320,35878.300780,36714.190830,35982.41365,46721.257740,3.573677e+04,3.934299e+04,5.080654e+04,4.376108e+04,6.499784e+04,3.977511e+04,4.465724e+04,4.823030e+04,4.713622e+04,3.309122e+04,3.527513e+04,3.611943e+04,3.478068e+04,3.437184e+04,3.449037e+04,3.585801e+04,3.686023e+04,3.585574e+04,3.637582e+04,3.669053e+04,3.739629e+04,3.830947e+04,4.002939e+04,4.087865e+04,4.165716e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albanie,ALB,Émissions totales de GES (kt d’équivalent CO2),EN.ATM.GHGT.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6960.868200,6950.036100,7613.266100,7348.49210,7632.664300,7873.037000,8470.978400,8948.980900,9996.056200,11248.358300,11433.584200,10520.362500,10730.526100,11567.573700,12215.237500,11821.583100,12079.10500,12175.483200,1.260389e+04,1.255274e+04,1.111273e+04,8.273632e+03,6.227153e+03,6.067180e+03,6.793796e+03,6.421571e+03,6.187384e+03,5.500607e+03,5.246131e+03,6.493487e+03,8.389277e+03,6.820624e+03,7.503060e+03,7.834016e+03,7.401102e+03,8.308386e+03,8.027150e+03,9.879394e+03,8.136785e+03,8.108812e+03,8.506996e+03,8.712296e+03,8.898641e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On va chercher à produire les premières statistiques descriptives, et ce notamment en traçant les courbes d'évolutions des PIB et les courbes d'évolution des émission de GES.

In [61]:
pib["Country name"]

KeyError: 'Country name'

In [ ]:
Bonjour comment ça va ? 